In [1]:
import os
import numpy as np
import pandas as pd
import shutil

## Remove Top_Output If Exists Else Create Newly

In [2]:
if "Top_Output" in next(os.walk(os.getcwd()))[1]:
    shutil.rmtree("Top_Output")

os.mkdir("Top_Output")

## Getting All Files List

In [3]:
cwd = os.getcwd() + "/"

directory_list = [cwd + "Top/"]
file_list = []

while len(directory_list) != 0:

    for dl in directory_list:

        dir_list = next(os.walk(dl))[1]
        for i in range(len(dir_list)):
            dir_list[i] = dl + dir_list[i] + "/"
            
        directory_list.extend(dir_list)

        file_list.extend([dl+fl for fl in next(os.walk(dl))[2]])

        directory_list.remove(dl)

## Extracting Data From The Files

In [4]:
data = {"File_Name": [],
        "Serial": [],
        "Litho_1": [],
        "Litho_2": [],
        "Exam_No": [],
        "Subject_Code": [],
        "Roll": [],
        "Venue_Code": []
       }

split_lenght = [0, 32, 32, 2, 3, 9, 3]
split = np.cumsum(split_lenght)

for fl in file_list:
    current_file = open(fl, "r")
    serial = 1
    for line in current_file:
        data["File_Name"].append(os.path.basename(fl)[:-4])
        data["Serial"].append(f"{serial:010d}")
        
        # iterate through all other keys excpet File_Name and Serial and set their values according to length
        key = list(data.keys())[2:]
        for i in range(0, len(key)):
            if key[i] == "Litho_1" or key[i] == "Litho_2":
                data[key[i]].append(line[split[i]:split[i+1]].replace(" ", "0"))    # replacing <space> with 0 in Litho
            else:
                data[key[i]].append(line[split[i]:split[i+1]])
        serial += 1

In [5]:
df = pd.DataFrame(data)

df.to_csv("Top_Output/All_Top.csv", index=False, sep=';')

## Seperate Duplicate Top Rows

In [6]:
duplicate_top = df[df.duplicated(["Litho_1", "Litho_2"])]

df = df.drop(duplicate_top.index)

duplicate_top.to_csv("Top_Output/Duplicate_Top.csv", index=False, sep=';')

## Reading Applicants Data

In [7]:
applicants = pd.read_csv("15th_ApplicantsInfo.txt", sep='|', encoding='latin1')

column_names = ["Roll", "Applicants_Subject_Code", "DoB(YMD)", "Invoice", "Subject_Name", "MCQ_Marks", "Applicants_Name", "Post_Code"]
applicants.columns = column_names

## Seperate Invalid Roll

In [8]:
applicants = applicants[["Roll", "Applicants_Subject_Code", "DoB(YMD)", "Applicants_Name"]]

applicants["Roll"] = applicants["Roll"].astype(str)
df["Roll"] = df["Roll"].astype(str)

tdf = pd.merge(df, applicants, on="Roll", how="inner", suffixes=("", "_Main"))

In [9]:
tdf_roll = list(tdf["Roll"])

invalid_roll = df[~df["Roll"].isin(tdf_roll)]

invalid_roll.to_csv("Top_Output/Invalid_Roll.csv", index=False, sep=';')

df = tdf.copy()

## Seperate Unmatch Litho

In [10]:
unmatched_litho = df[df["Litho_1"] != df["Litho_2"]]

df = df.drop(unmatched_litho.index)

unmatched_litho = unmatched_litho.sort_values(["File_Name"])
unmatched_litho["File_Name"][unmatched_litho["File_Name"].duplicated()] = ""

unmatched_litho.to_excel("Top_Output/Unmatched_Litho_Top.xlsx", index=False)

/home/hpdipto/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Seperate Duplicate Roll

In [11]:
duplicate_roll = df["Roll"].duplicated(keep=False)

duplicate_roll = df[duplicate_roll]

duplicate_roll = duplicate_roll.sort_values(["File_Name", "Roll"], ascending=[0, 0])

df = df.drop(duplicate_roll.index)

dup_roll = duplicate_roll[["File_Name", "Serial", "Roll"]]   # cosidering only three columns

In [12]:
dr = dup_roll["Roll"].unique()
duplicate_roll_file_list = {}

for r in dr:
    if r not in duplicate_roll_file_list:
        duplicate_roll_file_list[r] = []

In [13]:
get_file_info = lambda name: [(i, j) for i, j in zip(list(name["File_Name"]), list(name["Serial"]))]
file_name_list = dup_roll.groupby(["Roll"]).apply(get_file_info)

for r in duplicate_roll_file_list:
    duplicate_roll_file_list[r].extend(list(file_name_list[r]))

In [14]:
another_file_list = []

for i in range(len(dup_roll)):
    roll = dup_roll["Roll"].iloc[i]
    file_name = dup_roll["File_Name"].iloc[i]
    serial = dup_roll["Serial"].iloc[i]
    
    lst = duplicate_roll_file_list[roll].copy()
    lst2 = duplicate_roll_file_list[roll].copy()
    for l in lst2:
        if file_name in l:
            lst.remove(l)
            
    another_file_list.append(str(lst)[1:-1])
    
dup_roll["Another_Duplicate_File_List"] = another_file_list

/home/hpdipto/.local/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [15]:
machine_seperator = lambda name: 1 if "M01" in name else 2

fltr = dup_roll["File_Name"].apply(machine_seperator)
machine_1 = [True if i == 1 else False for i in fltr]
machine_2 = [True if i == 2 else False for i in fltr]

machine_1 = dup_roll[machine_1]
machine_2 = dup_roll[machine_2]

In [16]:
machine_1["File_Name"][machine_1.duplicated(["File_Name"])] = ""
machine_2["File_Name"][machine_2.duplicated(["File_Name"])] = ""

machine_1.to_excel("Top_Output/Duplicate_Roll_M01_Report.xlsx", index=False)
machine_2.to_excel("Top_Output/Duplicate_Roll_M02_Report.xlsx", index=False)

/home/hpdipto/.local/lib/python3.6/site-packages/pandas/core/generic.py:9116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/home/hpdipto/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3326: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


## Final File

In [17]:
df.to_csv("Top_Output/Top.csv", index=False, sep=';')